# Modello AI per MNIST digit recognition
Per installare le dipendenze necessarie definite in requirements.txt
```bash
pip install -r requirements.txt
```
oppure
```bash
pip install torch torch torchvision torchaudio onnx
```

In [ ]:
! pip install torch torch torchvision torchaudio onnx

In [ ]:
#Importo le librerie necessarie
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import cuda
import torch

In [ ]:
#Imposto il device su GPU se disponibile
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
#Scarico il dataset MNIST
dataset = datasets.MNIST('dataset', download=True, train=True, transform=ToTensor())

#Creo il dataloader
dataloader = DataLoader(dataset, batch_size=64)

## La rete neurale
Questo è la CNN (Convolutional Neural Network) che useremo.
È composta da layer che eseguiranno una convoluzione, la funzione di attivazione ReLU ed infine una funzione lineare che riduce l'output alle 10 cifre del dataset.
I layer che eseguono la convoluzione prendono in input un tensore (la nostra immagine) e applicano i filtri con una "finestra" 3\*3. Il valore dei filtri verra calcolato dalla rete neurale durante la fase di "backward propagation".
Il motivo per cui l'ultimo layer ha '128*(28-(2*3))*(28-(2*3))' come input in quanto il tensore è stato reso bidimensionale, ma ad ogni convoluzione allìimmagine è stato rimosso un pixel da ogni lato.
La funzione di attivazione compie un compito molto importante. Come in una rete neurale naturale i neuroni possono essere stimolati o no da uno stimolo esterno, così i neuroni digitali si "stimolano" in relazione alla funzione di attivazione. Inoltre prende parte nella fase di "backward propagation".

In [ ]:
class Model(nn.Module):
    #Constuttore
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, 3), # 32 filters (la parte che verrà addestrata) , 3x3 kernel
            nn.ReLU(), # Funzione di attivazione (rectified linear activation function)
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3),
            nn.Flatten(), # Appiattisco l'immagine
            nn.Linear(64*(28-(2*3))*(28-(2*3)), 10), # 128 neuroni, 10 classes
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
#Creo il modello e lo sposto sulla GPU se disponibile
model = Model().to(device)

## Loss function
La funzione di perdita (loss function), in questo caso `nn.CrossEntropyLoss()`, è usata durante l'addestramento di una rete neurale per misurare quanto bene il modello sta facendo le sue previsioni.

Quando viene chiamata `loss_fn(outputs, targets)`, calcola la log probabilità delle classi (i 10 numeri) previste (usando `LogSoftmax`) e poi calcola la `NLLLoss` (Negative Log Likehood Loss) tra le previsioni e i veri valori.

Durante l'addestramento, l'obiettivo è minimizzare questa funzione di perdita. Questo significa che si vuole ridurre la differenza tra ciò che il modello prevede e i valori reali. Quando la funzione di perdita è minima, il modello ha la migliore performance possibile sui dati di addestramento.

In [ ]:
#Definisco la funzione di perdita, buona per classificazione multiclasse
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# Optimizer
optimizer = Adam(model.parameters(), lr=1e-3)

In [ ]:
# Funzione di valutare il modello con dati che non ha mai visto
def evaluate(model):
    dataset = datasets.MNIST('dataset', download=True, train=False, transform=ToTensor())
    dataloader = DataLoader(dataset, batch_size=64)
    correct = 0
    total = len(dataset)
    for data, target in dataloader:
        data = data.to(device)
        target = target.to(device)
        pred = model(data)
        correct += (pred.argmax(1) == target).type(torch.float).sum().item()
    print(f'Accuracy: {(correct/total)*100}%')

In [ ]:
# Fuzione per esportare il modello in formato ONNX
def toONNX(model, filename):
    #                        (batch, channel, width and height)
    dummy_input = torch.randn(1, 1, 28, 28).to(device)
    torch.onnx.export(model, dummy_input, filename, verbose=True)

# Main Trainig Loop

In [ ]:
#Trainig loop
if __name__ == '__main__':
    for epoch in range(10): # 10 epochs
        for batch_idx, (data, target) in enumerate(dataloader): 
            data = data.to(device)
            target = target.to(device)

            # Forward
            pred = model(data).to(device)
            loss = loss_fn(pred, target)

            # Backward
            optimizer.zero_grad()
            loss.backward()

            # Update
            optimizer.step()

            if batch_idx % 100 == 0:
                print(f'Epoch: {epoch}, Loss: {loss.item()}')

    model.eval()

    torch.save(model.state_dict(), 'model.pth')

    print('Done training')

    evaluate(model)

    toONNX(model, 'model.onnx')